<a href="https://colab.research.google.com/github/Sharan-Babu/Deepfake/blob/master/Deepfakes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U ipykernel
!pip install -q streamlit
!pip install pyngrok

     |████████████████████████████████| 122kB 5.9MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.4 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


     |████████████████████████████████| 7.2MB 6.1MB/s 
     |████████████████████████████████| 4.4MB 39.0MB/s 
     |████████████████████████████████| 102kB 9.8MB/s 
     |████████████████████████████████| 112kB 44.3MB/s 
  Created wheel for pyngrok: filename=pyngrok-4.1.10-cp36-none-any.whl size=16377 sha256=b6e218eb4ca933009491d100645194f64faff3d87e41fa401e24cd91e05b396d
  Stored in directory: /root/.cache/pip/wheels/0d/7f/06/e8079dd6f80eb6683fac47c4e1cb720b61cd82d6e4c13d87a5
Successfully built pyngrok


In [1]:
!pip install PyYAML
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 243 (delta 0), reused 1 (delta 0), pack-reused 239
Receiving objects: 100% (243/243), 71.46 MiB | 17.62 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [ ]:
cd first-order-model

/content/first-order-model


In [ ]:
import sys
sys.path.append('first-order-model')

In [ ]:
!pwd

/content


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
from google.colab import files
upload1 = files.upload()

Saving vox-256.yaml to vox-256.yaml


In [8]:
upload2 = files.upload()

Saving 02.png to 02.png


In [5]:
!pwd

/content/first-order-model


In [3]:
cd first-order-model

/content/first-order-model


In [6]:
cd ..

/content


In [16]:
!ngrok authtoken 1gBC1BizajDqpzJUZq3DFx0MDLJ_3oikNo2K7U6Si2MX9mWCm

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [10]:
import sys
sys.path.insert(0, 'first-order-model')

In [11]:
import streamlit as st
#st.set_option('deprecation.showfileUploaderEncoding', False)
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

In [ ]:
#! ngrok authtoken <your_auth_token>

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [15]:
uploaded4 = files.upload()

Saving vox-256.yaml to vox-256.yaml


In [ ]:
cd ..

/content


In [1]:
%%writefile score.py
import streamlit as st
st.set_option('deprecation.showfileUploaderEncoding', False)
import imageio
import numpy as np
import sys
sys.path.insert(0, 'first-order-model')
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
#addedd vox.yaml

import warnings
warnings.filterwarnings("ignore")
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte
import io
import os
import base64

def get_binary_file_downloader_html(bin_file, file_label='File'):
    with open(bin_file, 'rb') as f:
        data = f.read()
    bin_str = base64.b64encode(data).decode()
    href = f'<a href="data:application/octet-stream;base64,{bin_str}" download="generated.mp4">Video Download Link</a>'
    return href

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

st.title("Deepfake Generator")
st.subheader("Produce synthetic videos with AI!")

st.header("Upload Image:")
st.subheader("Synthetic video will be produced from this image")
image_file = st.file_uploader("Enter Image path/name:",type=["png"])
st.text('')

st.header("Upload Video File:")
st.subheader("The movements of the person in this video will be used for producing the synthetic video.")
video_file = st.file_uploader("Enter Video path/name:",type=["mp4"])

if st.button('Generate Video!'):
    if image_file is None or video_file is None:
        st.markdown("# Please upload both Image and Video.")
    else:
        source_image = imageio.imread(image_file)
        st.text('You uploaded this image:')
        st.image(image_file,width=300)
        driving_video = imageio.mimread('00.mp4')

        with st.spinner('Processing the video...'):
            source_image = resize(source_image, (256,256))[..., :3]
            driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
            
            
        
            generator, kp_detector = load_checkpoints(config_path='vox-256.yaml', checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')
            predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
            #test = imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
            imageio.mimsave('generated.mp4', [img_as_ubyte(frame) for frame in predictions])
            st.video('generated.mp4')
            #st.video(test)
            
            st.markdown(get_binary_file_downloader_html('generated.mp4', 'Picture'), unsafe_allow_html=True)
        st.balloons()

Overwriting score.py


In [2]:
!nohup streamlit run score.py &

nohup: appending output to 'nohup.out'


In [3]:
from pyngrok import ngrok

url = ngrok.connect(port=8501)
url

'http://7e2b76fd1c8f.ngrok.io'

In [24]:
ngrok.disconnect(public_url="http://7fd8c38b567c.ngrok.io/")